## Imports and constants

In [88]:
# Standard Library
import random
from typing import Tuple, Iterator

# Third party
import numpy as np

# Visualization
import gif
import plotly.graph_objects as go
import plotly.figure_factory as ff


CELL_COLOR = "#fbab3a"
NEIGHBORHOOD_COLOR = "#f08080"
PLOT_COLORS = ("#9b76bc", "#618da7")
# PLOT_COLORS = ("#ffba08", "#ff8c61")

ArrayShape = Tuple[int, int]


class SeedArray:
    @staticmethod
    def __call__(shape: ArrayShape) -> np.ndarray:
        f = random.choice((SeedArray.binary_array,
                           SeedArray.diagonal,
                           SeedArray.inverted_diagonal,
                           SeedArray.quilt))
        return f(shape)

    @staticmethod
    def binary_array(shape: tuple) -> np.ndarray:
        return np.random.randint(0, 2, shape, dtype="uint8")

    @staticmethod
    def diagonal(shape: tuple) -> np.ndarray:
        # x, y = shape
        tri = np.triu(SeedArray.binary_array(shape))
        return np.clip(tri + tri.T, 0, 1)

    @staticmethod
    def inverted_diagonal(shape: tuple) -> np.ndarray:
        # x, y = shape
        tri = np.triu(SeedArray.binary_array(shape))
        return np.triu(np.where(tri, 0, 1)).T + tri

    @staticmethod
    def quilt(shape: tuple) -> np.ndarray:
        array = np.triu(SeedArray.binary_array(shape))
        rotator = random.choice((np.fliplr, np.flipud, np.rot90, None))
        if not rotator:
            return array
        return rotator(array)


class TileMethod:
    @staticmethod
    def __call__(array: np.ndarray, pattern_number: int) -> np.ndarray:
        tiling_method = random.choice((TileMethod.four_corners,
                                       TileMethod.book_match,
                                       TileMethod.hamburger,
                                       TileMethod.repeat))
        return np.tile(tiling_method(array), (pattern_number,)*2)

    @staticmethod
    def four_corners(NW: np.ndarray) -> np.ndarray:
        NE = np.fliplr(NW)
        SW = np.flipud(NW)
        SE = np.flipud(NE)
        return np.block([[NW, NE], [SW, SE]])

    @staticmethod
    def book_match(L: np.ndarray) -> np.ndarray:
        R = np.fliplr(L)
        return np.block([[L, R], [L, R]])

    @staticmethod
    def hamburger(T: np.ndarray) -> np.ndarray:
        B = np.flipud(T)
        return np.block([[T, T], [B, B]])

    @staticmethod
    def repeat(x: np.ndarray) -> np.ndarray:
        return np.tile(x, (2, 2))


class LifeSeedGenerator:
    def __init__(self, n: int):
        self.seed = self.generator(n)

    def __call__(self) -> np.ndarray:
        return next(self.seed)

    def generator(self, n: int) -> Iterator[np.ndarray]:
        while True:
            pattern_number, initial_size = self.size_and_pattern(n)
            shape = (initial_size,)*2
            array_method = SeedArray()
            tiling_method = TileMethod()
            yield tiling_method(array_method(shape), pattern_number)

    @staticmethod
    def size_and_pattern(n: int) -> ArrayShape:
        k = n // 2
        d = [(x, k//x) for x in range(2, k + 1) if k % x == 0]
        return sorted(random.choice(d))


def annotated_heatmap(z: np.ndarray) -> go.Figure:
    fig = ff.create_annotated_heatmap(z=z,
                                      font_colors=["white"],
                                      coloraxis="coloraxis")
    return update_heatmap(fig, shape=z.shape)


def plain_heatmap(z: np.ndarray) -> go.Figure:
    fig = go.Figure(go.Heatmap(z=z, coloraxis="coloraxis"))
    return update_heatmap(fig, shape=z.shape)


def update_heatmap(fig: go.Figure, shape: tuple = (70, 70)) -> go.Figure:
    height, width = shape
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.update_layout(height=height*10,
                      width=width*10,
                      margin_t=0,
                      margin_b=0,
                      margin_l=0,
                      margin_r=0,
                      showlegend=False,
                      paper_bgcolor="rgba(0, 0, 0, 0)",
                      plot_bgcolor="rgba(0, 0, 0, 0)",
                      coloraxis={"colorscale": PLOT_COLORS,
                                 "showscale": False})
    return fig


def add_rect(fig: go.Figure, *, rect_params: dict, opacity: float = 0.0) -> go.Figure:
    outline_rect = {"type": "rect", "line_width": 5, **rect_params}
    shapes = fig.layout.shapes
    if opacity:
        filled_rect = {**outline_rect, "fillcolor": outline_rect["color"], "opacity": opacity}
        shapes += (filled_rect,)
    shapes += (outline_rect,)
    fig.update_layout(shapes=shapes)
    return fig


def add_line(fig: go.Figure, *, color, dims) -> go.Figure:
    line = {"type": "line", "line_width": 5, "line_color": color, **dims}
    shapes = fig.layout.shapes + (line,)
    fig.update_layout(shapes=shapes)
    return fig


@gif.frame
def make_frame(x):
    return plain_heatmap(x)


def make_animation(seed_gen):
    frame0 = seed_gen()
    x, _ = frame0.shape
    frames = [make_frame(frame0)] + [make_frame(seed_gen()) for _ in range(100)]
    gif.save(frames, f"{x}x{x}_symmetries.gif", duration=500)
    

def make_neighborhood_plot(x, *rects):
    fig = annotated_heatmap(x)
    for rect in rects:
        fig = add_rect(fig, rect_params=rect)
    return update_heatmap(fig)


In [119]:
x = np.random.randint(0, 2, (10, 10))

In [120]:
# Normal neighborhood
neighborhood = dict(x0=4.5, x1=7.5, y0=2.5, y1=5.5, line_color=NEIGHBORHOOD_COLOR)
cell = dict(x0=5.5, x1=6.5, y0=3.5, y1=4.5, line_color=CELL_COLOR)

make_neighborhood_plot(x, cell, neighborhood)

In [85]:
# Fixed boundaries edge neighborhood
original_dims = dict(x0=0.5, x1=10.5, y0=0.5, y1=10.5, line_color="#70c2b4")
neighborhood = dict(x0=-0.5, x1=2.5, y0=7.5, y1=4.5, line_color=NEIGHBORHOOD_COLOR)
cell = dict(x0=0.5, x1=1.5, y0=6.5, y1=5.5, line_color=CELL_COLOR)

make_neighborhood_plot(np.pad(x, 1), original_dims, neighborhood, cell)

In [101]:
# Periodic boundaries edge neighborhood
cell = dict(x0=-0.5, x1=0.5, y0=-0.5, y1=0.5, line_color=CELL_COLOR)
lines = [dict(x0=-0.5, x1=1.5, y0=1.5, y1=1.5),
         dict(x0=-0.5, x1=1.5, y0=8.5, y1=8.5),
         dict(x0=8.5, x1=9.5, y0=8.5, y1=8.5),
         dict(x0=1.5, x1=1.5, y0=8.5, y1=9.5),
         dict(x0=1.5, x1=1.5, y0=1.5, y1=-0.5),
         dict(x0=8.5, x1=8.5, y0=1.5, y1=-0.5),
         dict(x0=8.5, x1=9.5, y0=1.5, y1=1.5),
         dict(x0=8.5, x1=8.5, y0=8.5, y1=9.5)]

fig = make_neighborhood_plot(x, cell)

for line in lines:
    fig = add_line(fig, color=NEIGHBORHOOD_COLOR, dims=line)

fig = update_heatmap(fig)
fig.show()

In [94]:
# x = np.random.randint(0, 2, (5, 5))

# pad_x = np.pad(x, 1)
tiled_x = np.tile(x, (3, 3))

# pad_dims = dict(x0=0.5, x1=10.5, y0=0.5, y1=10.5)
# tiled_dims = dict(x0=4.5, x1=9.5, y0=4.5, y1=9.5)
slice_dims = dict(x0=3.5, x1=10.5, y0=3.5, y1=10.5)

# fig = heatmap(tiled_x)
fig = add_rect(fig, CELL_COLOR, 0, **slice_dims)
fig.show()

In [117]:
# x = np.random.randint(0, 2, (3, 3))

# annotations = [["(r + 1, c - 1)", "(r + 1, c)", "(r + 1, c + 1)"],
#                ["(r, c - 1)", "(r, c)", "(r, c + 1)"],
#                ["(r - 1, c - 1)", "(r - 1, c)", "(r - 1, c + 1)"]]

annotations = [["alive" if x[i,j] == 1 else "dead" for j in range(3)] for i in range(3)]


fig = ff.create_annotated_heatmap(z=x, annotation_text=annotations, font_colors=["white"], coloraxis="coloraxis")
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(height=300,
                  width=300,
                  margin_t=0,
                  margin_b=0,
                  margin_l=0,
                  margin_r=0,
                  showlegend=False,
                  coloraxis={"colorscale": PLOT_COLORS,
                             "showscale": False})


# neighborhood = dict(x0=-0.5, x1=2.5, y0=-0.5, y1=2.5)
cell = dict(x0=0.5, x1=1.5, y0=0.5, y1=1.5, line_color=CELL_COLOR)
fig = add_rect(fig, rect_params=cell)
# fig = add_rect(fig, NEIGHBORHOOD_COLOR, 0, **neighborhood)

fig.show()

In [147]:
# f = heatmap(np.random.multinomial(1, [0.25, 0.75], size=(10, 10)))
# b1, b2 = np.random.binomial(1, 0.5, (2, 100, 100))
# f = heatmap(np.bitwise_and(np.bitwise_and(b1, b2), np.bitwise_or(b1, b2)))
# f.show()
x = np.random.normal(scale=5, size=(100, 100))
x_mean = x.mean()
x_std = 0.25 * x.std()
low, high = x_mean - x_std, x_mean + x_std


heatmap(np.where((low < x) & (x < high), 1, 0)).show()

In [27]:
# data processing
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

# plotting
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# image processing
import gif


BG_COLOR = "#696969"
CELLS_COLOR = "#f08080"
BOARD_COLOR = "#696969"
SELECT_3x3_COLOR = "#70c2b4"
SELECT_1x1_COLOR = "#fbab3a"
PLOT_COLORS = ("#9b76bc", "#618da7")


DIMS = {
    0: {"x0": -0.5, "y0": 4.5, "x1": 2.5, "y1": 1.5},
    1: {"x0": 0.5, "y0": 4.5, "x1": 3.5, "y1": 1.5},
    2: {"x0": 1.5, "y0": 4.5, "x1": 4.5, "y1": 1.5},
    3: {"x0": -0.5, "y0": 3.5, "x1": 2.5, "y1": 0.5},
    4: {"x0": 0.5, "y0": 3.5, "x1": 3.5, "y1": 0.5},
    5: {"x0": 1.5, "y0": 3.5, "x1": 4.5, "y1": 0.5},
    6: {"x0": -0.5, "y0": 2.5, "x1": 2.5, "y1": -0.5},
    7: {"x0": 0.5, "y0": 2.5, "x1": 3.5, "y1": -0.5},
    8: {"x0": 1.5, "y0": 2.5, "x1": 4.5, "y1": -0.5}
}


CENTRAL_DIMS = {
    0: {"x0": 0.5, "y0": 3.5, "x1": 1.5, "y1": 2.5},
    1: {"x0": 1.5, "y0": 3.5, "x1": 2.5, "y1": 2.5},
    2: {"x0": 2.5, "y0": 3.5, "x1": 3.5, "y1": 2.5},
    3: {"x0": 0.5, "y0": 2.5, "x1": 1.5, "y1": 1.5},
    4: {"x0": 1.5, "y0": 2.5, "x1": 2.5, "y1": 1.5},
    5: {"x0": 2.5, "y0": 2.5, "x1": 3.5, "y1": 1.5},
    6: {"x0": 0.5, "y0": 1.5, "x1": 1.5, "y1": 0.5},
    7: {"x0": 1.5, "y0": 1.5, "x1": 2.5, "y1": 0.5},
    8: {"x0": 2.5, "y0": 1.5, "x1": 3.5, "y1": 0.5}
}


REFS = {
    0: {"xref": "x16", "yref": "y16"},
    1: {"xref": "x17", "yref": "y17"},
    2: {"xref": "x18", "yref": "y18"},
    3: {"xref": "x10", "yref": "y10"},
    4: {"xref": "x11", "yref": "y11"},
    5: {"xref": "x12", "yref": "y12"},
    6: {"xref": "x4", "yref": "y4"},
    7: {"xref": "x5", "yref": "y5"},
    8: {"xref": "x6", "yref": "y6"}
}

## Functions

In [ ]:
def get_annotations(x):
    n = len(x)
    annotation_lookup = dict(zip(range(10), [""] + list("ABCDEFGHI")))
    return [[annotation_lookup[x[j, i]] for i in range(n)] for j in range(n)]


def update_annotations(fig, x, y):
    for annotation in fig["layout"]["annotations"]:
        annotation["xref"] = x
        annotation["yref"] = y
    return fig


def make_heatmap(z):
    return ff.create_annotated_heatmap(z=z,
                                       font_colors=["white"],
                                       coloraxis="coloraxis")


@gif.frame
def make_sliding_window_plot(big_fig, windows, *, dims, central_dims, refs):
    specs = [[{"colspan": 3, "rowspan": 3}] + [{}]*5, [{}]*6, [{}]*6]
    fig = make_subplots(rows=3,
                        cols=6,
                        specs=specs,
                        horizontal_spacing=0.015,
                        vertical_spacing=0.015,
                        start_cell="bottom-left",
                        print_grid=False)
    line_width = 6
    opacity = 0.2
    annotations = []
    big_fig = update_annotations(big_fig, "x", "y")
    annotations.extend(big_fig.layout.annotations)
    fig.add_trace(big_fig.data[0], row=1, col=1)

    for i, row in enumerate((1, 2, 3)):
        for j, col in enumerate((4, 5, 6)):
            window_refs = REFS[6 - 3*i + j]
            xref, yref = window_refs["xref"], window_refs["yref"]
            window_fig = make_heatmap(windows[i, j])
            window_fig = update_annotations(window_fig, xref, yref)
            annotations.extend(window_fig.layout.annotations)
            fig.add_trace(window_fig.data[0], row=row, col=col)

    shapes = []
    for i in range(9):
        window_refs = REFS[i]
        xref, yref = window_refs["xref"], window_refs["yref"]
        shapes.append({"type": "rect",
                       "x0": -0.5,
                       "y0": 2.5,
                       "x1": 2.5,
                       "y1": -0.5,
                       "xref": xref,
                       "yref": yref,
                       "line_width": line_width,
                       "line_color": BOARD_COLOR})

    # TODO: implement shape maker helper function
    shapes += [{"type": "rect",
                "x0": -0.5,
                "y0": 4.5,
                "x1": 4.5,
                "y1": -0.5,
                "line_width": line_width,
                "line_color": BOARD_COLOR},
               {"type": "rect",
                "x0": 0.5,
                "y0": 3.5,
                "x1": 3.5,
                "y1": 0.5,
                "line_width": line_width,
                "line_color": CELLS_COLOR},
               {"type": "rect",
                "xref": "x",
                "yref": "y",
                **dims,
                "line_width": line_width,
                "line_color": SELECT_3x3_COLOR,
                "fillcolor": SELECT_3x3_COLOR,
                "opacity": opacity},
               {"type": "rect",
                "xref": "x",
                "yref": "y",
                **central_dims,
                "line_width": line_width,
                "line_color": SELECT_1x1_COLOR,
                "fillcolor": SELECT_1x1_COLOR,
                "opacity": opacity},
               {"type": "rect",
                "xref": "x",
                "yref": "y",
                **dims,
                "line_width": line_width,
                "line_color": SELECT_3x3_COLOR},
               {"type": "rect",
                "xref": "x",
                "yref": "y",
                **central_dims,
                "line_width": line_width,
                "line_color": SELECT_1x1_COLOR},
               {"type": "rect",
                "x0": -0.5,
                "y0": 2.5,
                "x1": 2.5,
                "y1": -0.5,
                **refs,
                "line_width": line_width,
                "line_color": SELECT_3x3_COLOR,
                "fillcolor": SELECT_3x3_COLOR,
                "opacity": opacity},
               {"type": "rect",
                "x0": 0.5,
                "y0": 1.5,
                "x1": 1.5,
                "y1": 0.5,
                **refs,
                "line_width": line_width,
                "line_color": SELECT_1x1_COLOR,
                "fillcolor": SELECT_1x1_COLOR,
                "opacity": opacity},
               {"type": "rect",
                "x0": -0.5,
                "y0": 2.5,
                "x1": 2.5,
                "y1": -0.5,
                **refs,
                "line_width": line_width,
                "line_color": SELECT_3x3_COLOR},
               {"type": "rect",
                "x0": 0.5,
                "y0": 1.5,
                "x1": 1.5,
                "y1": 0.5,
                **refs,
                "line_width": line_width,
                "line_color": SELECT_1x1_COLOR}]

    fig.update_layout(height=500,
                      width=950,
                      shapes=shapes,
                      margin_t=10,
                      margin_b=10,
                      margin_l=10,
                      margin_r=10,
                      showlegend=False,
                      paper_bgcolor=BG_COLOR,
                      plot_bgcolor=BG_COLOR,
                      coloraxis={"colorscale": PLOT_COLORS,
                                 "showscale": False})
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.layout.annotations = annotations
    return fig


def make_sliding_window_animation(cells, boundary="fixed"):
    if boundary == "fixed":
        game_board = np.pad(cells, 1)
    else:
        n = len(cells) - 1
        game_board = np.tile(cells, (3, 3))[n:-n, n:-n]

    big_fig = make_heatmap(game_board)
    windows = sliding_window_view(game_board, (3, 3))

    frames = []
    for i in range(9):
        fig = make_sliding_window_plot(big_fig,
                                       windows,
                                       dims=DIMS[i],
                                       central_dims=CENTRAL_DIMS[i],
                                       refs=REFS[i])
        frames.append(fig)

    gif.save(frames, f"{boundary}_sliding_window_animation.gif", 750)

In [ ]:
cells = np.random.randint(0, 2, (3, 3))

# n = len(cells) - 1
# tiled_cells = np.tile(cells, (3, 3))
# periodic_boundary = sliding_window_view(tiled_cells[n:-n, n:-n], (3, 3))
# game_board, windows = tiled_cells, periodic_boundary

# make_heatmap(game_board)
make_sliding_window_animation(cells, boundary="fixed")
make_sliding_window_animation(cells, boundary="periodic")